In [167]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [148]:
spyTicker = yf.Ticker("SPY") # SPDR S&P 500 ETF Trust
vixTicker = yf.Ticker("^VIX") # VIX
df_spy = spyTicker.history(period="max")
df_vix = vixTicker.history(period="max")

In [149]:
#Explore Dataset
df_spy.index = df_spy.index.date

In [150]:
df_spy

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
1993-01-29,24.330336,24.330336,24.209289,24.313044,1003200,0.0,0.0,0.0
1993-02-01,24.330316,24.485947,24.330316,24.485947,480500,0.0,0.0,0.0
1993-02-02,24.468661,24.555123,24.416784,24.537830,201300,0.0,0.0,0.0
1993-02-03,24.572424,24.814516,24.555131,24.797224,529400,0.0,0.0,0.0
1993-02-04,24.883699,24.952869,24.607021,24.900991,531500,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
2025-11-10,677.239990,682.179993,675.030029,681.440002,75842900,0.0,0.0,0.0
2025-11-11,679.950012,683.570007,678.729980,683.000000,58953400,0.0,0.0,0.0
2025-11-12,684.789978,684.960022,680.950012,683.380005,62312500,0.0,0.0,0.0
2025-11-13,680.500000,680.859985,670.520020,672.039978,103457800,0.0,0.0,0.0


In [151]:
df_vix = df_vix.drop(df_vix.loc['1990':'1993-01-28'].index)

In [152]:
df_vix.index = df_vix.index.date

In [157]:
#join dataset
dataset = pd.merge(df_spy, df_vix, left_index=True, right_index=True, suffixes=('_SPY','_VIX'))
dataset = dataset.drop(['Dividends_SPY','Stock Splits_SPY', 'Capital Gains', 'Dividends_VIX', 'Stock Splits_VIX', 'Volume_VIX'], axis=1)


In [158]:
dataset

,Open_SPY,High_SPY,Low_SPY,Close_SPY,Volume_SPY,Open_VIX,High_VIX,Low_VIX,Close_VIX
1993-01-29,24.330336,24.330336,24.209289,24.313044,1003200,12.490000,13.160000,12.420000,12.420000
1993-02-01,24.330316,24.485947,24.330316,24.485947,480500,12.510000,12.920000,12.180000,12.330000
1993-02-02,24.468661,24.555123,24.416784,24.537830,201300,12.470000,12.890000,12.220000,12.250000
1993-02-03,24.572424,24.814516,24.555131,24.797224,529400,11.980000,12.340000,11.790000,12.120000
1993-02-04,24.883699,24.952869,24.607021,24.900991,531500,11.860000,12.840000,11.690000,12.290000
...,...,...,...,...,...,...,...,...,...
2025-11-10,677.239990,682.179993,675.030029,681.440002,75842900,18.580000,18.820000,17.600000,17.600000
2025-11-11,679.950012,683.570007,678.729980,683.000000,58953400,17.900000,18.010000,17.250000,17.280001
2025-11-12,684.789978,684.960022,680.950012,683.380005,62312500,17.209999,18.059999,17.100000,17.510000
2025-11-13,680.500000,680.859985,670.520020,672.039978,103457800,17.610001,21.309999,17.510000,20.000000


In [159]:
# SPY features
# daily log price and  natural log-returns for SPY
dataset["log_close_SPY"] = np.log(dataset["Close_SPY"])
dataset["log_open_SPY"] = np.log(dataset["Open_SPY"])
dataset["ret_1d"] = dataset["log_close_SPY"].diff(1)   # daily natural log-return

# intraday turbulance
dataset["intraday_OC"] = np.abs(dataset["log_open_SPY"] - dataset["log_close_SPY"])
dataset["intraday_CO"] = np.abs(dataset["log_close_SPY"].shift(1) - dataset["log_open_SPY"]) #What happend overnight? price jumped/Crashed? 
dataset["intraday_HL"] = np.log(dataset["High_SPY"]) - np.log(dataset["Low_SPY"])
dataset["intraday_HL_abs"] = np.abs(np.log(dataset["High_SPY"]) - np.log(dataset["Low_SPY"]))

# lagged returns (1 and 2 days)
dataset["ret_lag1"] = dataset["ret_1d"].shift(1)
dataset["ret_lag2"] = dataset["ret_1d"].shift(2)
dataset["intraday_HL_abs_lag1"] = dataset["intraday_HL_abs"].shift(1)
dataset["intraday_HL_abs_lag2"] = dataset["intraday_HL_abs"].shift(2)

# rolling statistics of returns
dataset["ret_roll_mean_5d"] = dataset["ret_1d"].rolling(5, min_periods=1).mean()
dataset["ret_roll_std_5d"] = dataset["ret_1d"].rolling(5, min_periods=1).std()
dataset["ret_roll_abs_5d"] = np.abs(dataset["ret_1d"].rolling(5, min_periods=1).sum())

dataset["ret_roll_mean_22d"] = dataset["ret_1d"].rolling(22, min_periods=1).mean()
dataset["ret_roll_std_22d"] = dataset["ret_1d"].rolling(22, min_periods=1).std()
dataset["ret_roll_abs_22d"] = np.abs(dataset["ret_1d"].rolling(22, min_periods=1).sum())

dataset["intraday_roll_CO_mean_5d"] = dataset["intraday_CO"].rolling(5, min_periods=1).mean()
dataset["intraday_roll_CO_std_5d"] = dataset["intraday_CO"].rolling(5,min_periods=1).std()
dataset["intraday_roll_CO_mean_22d"] = dataset["intraday_CO"].rolling(22,min_periods=1).mean()
dataset["intraday_roll_CO_std_22d"] = dataset["intraday_CO"].rolling(22,min_periods=1).std()


# Technical indicators: moving averages on Close (5, 10, 22 days)
dataset["ma_5"] = dataset["log_close_SPY"].rolling(window=5, min_periods=1).mean()
dataset["ma_10"] = dataset["log_close_SPY"].rolling(window=10,min_periods=1).mean()
dataset["ma_22"] = dataset["log_close_SPY"].rolling(window=22,min_periods=1).mean()

dataset["momentum_5d"] = dataset["log_close_SPY"] - dataset["log_close_SPY"].shift(5)
dataset["momentum_22d"] = dataset["log_close_SPY"] - dataset["log_close_SPY"].shift(22)

delta = dataset["ret_1d"]

gain = delta.clip(lower=0)  
loss = -delta.clip(upper=0) 

avg_gain = gain.rolling(5, min_periods=1).mean()
avg_loss = loss.rolling(5, min_periods=1).mean()

# Avoid divide-by-zero
avg_loss = avg_loss.replace(0, 1e-10)

RS = avg_gain / avg_loss
RSI = 100 - (100 / (1 + RS))

dataset["RSI_5d"] = RSI



# Rate of change (ROC) – 5 and 10 day
dataset["roc_5"] = dataset["Close_SPY"].pct_change(periods=5)
dataset["roc_10"] = dataset["Close_SPY"].pct_change(periods=10)

# Volume-related feature (log-volume + 10-day rolling mean)
dataset["log_volume"] = np.log(dataset["Volume_SPY"].replace(0, np.nan))
dataset["vol_roll_mean_10"] = dataset["log_volume"].rolling(window=10,min_periods=1).mean()

# Realized variance and lagged relalized variance daily, weekly, monthly
dataset["rvar_1d"] = dataset["ret_1d"] ** 2
dataset["rvar_5d"] = dataset["rvar_1d"].rolling(window=5,min_periods=1).sum()
dataset["rvar_22d"] = dataset["rvar_1d"].rolling(window=22,min_periods=1).sum()

# Realized volatility and lagged relalized volatility daily, weekly, monthly
dataset["rvol_1d"] = np.sqrt(dataset["rvar_1d"]) # Y lable
dataset["rvol_5d"] = np.sqrt(dataset["rvar_5d"])
dataset["rvol_22d"] = np.sqrt(dataset["rvar_22d"])



In [ ]:
dataset = dataset.dropna()

In [160]:
dataset

,Open_SPY,High_SPY,Low_SPY,Close_SPY,Volume_SPY,Open_VIX,High_VIX,Low_VIX,Close_VIX,log_close_SPY,...,roc_5,roc_10,log_volume,vol_roll_mean_10,rvar_1d,rvar_5d,rvar_22d,rvol_1d,rvol_5d,rvol_22d
1993-03-03,24.900975,24.987437,24.866390,24.970144,280100,13.140000,13.710000,12.930000,13.130000,3.217681,...,0.019775,0.038849,12.542902,11.702639,1.733962e-05,0.000248,0.001409,0.004164,0.015753,0.037530
1993-03-04,25.004716,25.004716,24.831793,24.831793,89500,12.520000,14.020000,12.500000,13.440000,3.212125,...,0.011980,0.033836,11.401994,11.558547,3.087004e-05,0.000275,0.001389,0.005556,0.016570,0.037271
1993-03-05,24.866413,24.970167,24.745367,24.762659,40000,13.200000,14.130000,12.980000,14.080000,3.209337,...,0.007742,0.027260,10.596635,11.572186,7.772750e-06,0.000280,0.001392,0.002788,0.016743,0.037316
1993-03-08,24.814511,25.315989,24.814511,25.315989,50800,15.000000,16.299999,14.950000,16.219999,3.231436,...,0.033169,0.046461,10.835652,11.340831,4.883799e-04,0.000761,0.001770,0.022099,0.027582,0.042074
1993-03-09,25.264125,25.281417,25.177663,25.229540,169300,14.020000,15.430000,13.780000,14.170000,3.228016,...,0.014604,0.043634,12.039428,11.261653,1.170070e-05,0.000556,0.001765,0.003421,0.023581,0.042006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-10,677.239990,682.179993,675.030029,681.440002,75842900,18.580000,18.820000,17.600000,17.600000,6.524208,...,-0.002780,-0.005545,18.144175,18.163432,2.397489e-04,0.000511,0.002111,0.015484,0.022611,0.045940
2025-11-11,679.950012,683.570007,678.729980,683.000000,58953400,17.900000,18.010000,17.250000,17.280001,6.526495,...,0.011492,-0.005909,17.892258,18.158817,5.228767e-06,0.000374,0.001365,0.002287,0.019346,0.036946
2025-11-12,684.789978,684.960022,680.950012,683.380005,62312500,17.209999,18.059999,17.100000,17.510000,6.527051,...,0.008560,-0.005834,17.947673,18.126998,3.093822e-07,0.000363,0.001133,0.000556,0.019043,0.033667
2025-11-13,680.500000,680.859985,670.520020,672.039978,103457800,17.610001,21.309999,17.510000,20.000000,6.510318,...,0.002581,-0.011459,18.454674,18.157400,2.800017e-04,0.000526,0.001412,0.016733,0.022940,0.037576


# Preprocessing for Random Forest Model

In [164]:
X = dataset
y = dataset["rvol_1d"].shift(-1)

print("shape of x:", X.shape)
print("shape of y:", y.tail())
X = X.iloc[:-1]
y = y.iloc[:-1]


shape of x: (8235, 46)
shape of y: 2025-11-10    0.002287
2025-11-11    0.000556
2025-11-12    0.016733
2025-11-13    0.000164
2025-11-14         NaN
Name: rvol_1d, dtype: float64


In [165]:
print("shape of x:", X.shape)
print("shape of y:", y.shape)
print("shape of y:", y.tail())

shape of x: (8234, 46)
shape of y: (8234,)
shape of y: 2025-11-07    0.015484
2025-11-10    0.002287
2025-11-11    0.000556
2025-11-12    0.016733
2025-11-13    0.000164
Name: rvol_1d, dtype: float64


In [169]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

In [172]:
# Initialize model
rf = RandomForestRegressor(
    n_estimators=1000, 
    max_depth=8,    
    random_state=42
)

# Fit on training data
rf.fit(X_train, y_train)

# Predict on test data
y_pred = rf.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
print("RMSE:", rmse)

RMSE: 0.009540135662298265
